# Notebook project about testing SAM to segment Cardiac MRI scans

## Google Colab configuration

In [ ]:
# change working directory

import os 

os.chdir("Malware-classification-model")
os.getcwd()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Copy your Kaggle API to /root/.config/kaggle and /root/.kaggle/kaggle.json

os.makedirs('/root/.kaggle', exist_ok = True)

!cp /content/drive/MyDrive/Kaggle_api/kaggle.json /root/.config/kaggle.json
!cp /content/drive/MyDrive/Kaggle_api/kaggle.json /root/.kaggle/kaggle.json

## Import Libraries

In [1]:
# If you're running this repository locally, RUN this cell:

import os
os.chdir("..")
os.getcwd()

'/Users/silvanoquarto/Desktop/LAVORO/MEDICAL_PHYSICS/Med-Physics'

In [20]:
# Import libraries 

import kaggle
import urllib.request
from pathlib import Path
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import torch
from segment_anything import sam_model_registry, SamPredictor
import h5py

## Setup and Configuration

In [9]:
# Set up device to use

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {DEVICE}")

Using device: cpu


In [10]:
# Create directories

data_dir = Path('data')
data_dir.mkdir(exist_ok=True)

## Download dataset

In [ ]:
def download_kaggle_dataset(dataset_name : str = None, kaggle_url : str = None):
    """
    Download ACDC dataset from Kaggle using Kaggle API.
    Requires:
    1. Kaggle account
    2. API token (kaggle.json) in ~/.kaggle/
    3. kaggle package installed: pip install kaggle
    """

    data_dir = Path('data')
    data_dir.mkdir(exist_ok=True)

    print(f"Downloading {dataset_name} dataset from Kaggle...")
    try:
        kaggle.api.authenticate()
        kaggle.api.dataset_download_files(
            kaggle_url,
            path=data_dir,
            unzip=True
        )
        print("Dataset downloaded and extracted successfully!")
        
    except Exception as e:
        print(f"Error downloading dataset: {e}")
        print("\nPlease ensure you have:")
        print("1. Created a Kaggle account")
        print("2. Generated an API token from https://www.kaggle.com/settings")
        print("3. Placed kaggle.json in ~/.kaggle/")
        print("4. Set appropriate permissions: chmod 600 ~/.kaggle/kaggle.json")
        raise

In [ ]:
# Download ACDC dataset from Kaggle using API key

download_kaggle_dataset(dataset_name="ACDC", kaggle_url='anhoangvo/acdc-dataset')

Dataset URL: https://www.kaggle.com/datasets/anhoangvo/acdc-dataset
Dataset downloaded and extracted successfully!


## Cardiac MRI Segmentation with SAM - Exploratory Analysis

In [13]:
# Set up SAM Model

def setup_sam():
    """Initialize and load SAM model."""

    sam_checkpoint = "sam_vit_b_01ec64.pth"
    checkpoint_url = "https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth"

    if not os.path.exists(sam_checkpoint):
        print("Downloading SAM checkpoint...")
        urllib.request.urlretrieve(checkpoint_url, sam_checkpoint)

    model_type = "vit_b"
    sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
    sam.to(device=DEVICE)

    return sam

In [16]:
sam = setup_sam()
predictor = SamPredictor(sam)

/Users/silvanoquarto/Desktop/LAVORO/MEDICAL_PHYSICS/Med-Physics/.venv/lib/python3.11/site-packages/segment_anything/build_sam.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feat

## Data Loading & Preprocessing